# Тестирование

## T5

In [28]:
import pandas as pd
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
from tqdm.auto import tqdm, trange
import gc
import csv

Выбираем предобученную модель _T5_ и выбираем предобученный токенизатор:

In [29]:
model = T5ForConditionalGeneration.from_pretrained(r'weights/t5_small_gsd_kmeans__results/t5_small_cl_train_38000')

base_model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Прописываем функцию для генерации предсказания модели:

In [30]:
def generate(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    
    result = model.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, 
        temperature=temperature, 
        max_length=max_length,
        repetition_penalty=3.0,
        num_beams=beams,
    )
    
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    
    if not n and isinstance(text, str):
        return texts[0]
    return texts

Посмотрим несколько примеров:

In [33]:
print(generate(['оставаться'], model, temperature=30.0, beams=10))

['оратс,VERB,Imp,Ind,S']


In [36]:
print(generate(['балка'], model, temperature=30.0, beams=10))

['алка,NOUN,Inan,Nom,F']


Сделаем предсказание для каждого слова в тестовом датасете:

In [37]:
path_to_test = r'data/prepared_data/GSD/gsd_test_data.csv'

In [38]:
old_df = pd.read_csv(path_to_test)
old_df

,form,data
0,Билли,"Билли,PROPN,Anim,Nom,Masc,Sing"
1,начал,"начать,VERB,Perf,Masc,Ind,Sing,Past,Fin,Act"
2,играть,"играть,VERB,Imp,Inf,Act"
3,за,"за,ADP,None"
4,резервный,"резервный,ADJ,Inan,Acc,Pos,Masc,Sing"
...,...,...
11380,Землю,"земля,NOUN,Inan,Acc,Fem,Sing"
11381,ради,"ради,ADP,None"
11382,получения,"получение,NOUN,Inan,Gen,Neut,Sing"
11383,энергии,"энергия,NOUN,Inan,Gen,Fem,Sing"


In [39]:
forms = old_df['form'].to_list()
forms

['Билли',
 'начал',
 'играть',
 'за',
 'резервный',
 'состав',
 '``',
 'Черка',
 '&#39;&#39;',
 'в',
 'возрасте',
 '16',
 'лет',
 ',',
 'а',
 'через',
 'пару',
 'сезонов',
 'был',
 'приглашён',
 'в',
 'основной',
 'состав',
 '.',
 'Стоимость',
 'проезда',
 'с',
 '5',
 'января',
 '2013',
 'года',
 '--',
 '15',
 'рублей',
 ',',
 'движение',
 'осуществляется',
 'с',
 '6.00',
 'до',
 '00.20',
 '.',
 'Стал',
 'членом',
 'секретариата',
 'общественной',
 'безопасности',
 'Мексики',
 '(',
 'SSP',
 ',',
 'Secretaría',
 'de',
 'Seguridad',
 'Pública',
 ')',
 'и',
 'специальным',
 'уполномоченным',
 'Федеральной',
 'полиции',
 'Мексики',
 '(',
 'PFP',
 ',',
 'Policía',
 'Federal',
 'Preventiva',
 ')',
 '.',
 'Официальный',
 'код',
 'округа',
 'AT125',
 '.',
 'Здесь',
 'обитает',
 'несколько',
 'десятков',
 'видов',
 'птиц',
 '.',
 'Когда',
 'Мэлоуну',
 'исполнилось',
 'три',
 'года',
 ',',
 'его',
 'отец',
 ',',
 'Шедрик',
 'Хэй',
 ',',
 'ушел',
 'из',
 'семьи',
 'и',
 'женился',
 'на',
 'другой

In [12]:
%%time
predictions = []
for form in forms:
    pred = generate(form, model, temperature=50.0, beams=10)
    predictions.append(pred)

CPU times: total: 8h 18min 39s
Wall time: 1h 2min 42s


In [14]:
old_df['predicted_data'] = predictions
old_df.to_csv(r'testing/gsd_test_t5_kmeans.csv')

## RuPrompts

In [15]:
from transformers import GPT2LMHeadModel, AutoTokenizer
from transformers import pipeline
from ruprompts import Prompt

Выбираем предобученную модель, которая использовалась в обучении:

In [16]:
backbone_id = "sberbank-ai/rugpt3large_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Выбираем нужный нам checkpoint для тестирования модели:

In [17]:
prompt = Prompt.from_pretrained(r"weights/rupr_not_clustered_syn/checkpoint-80850")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer)

Посмотрим пример работы генерации признаков для нового слова:

In [18]:
ppln({"form": "Ублюдок"}, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'NOUN,Anim,Nom,Masc,Sing'}]

Выбираем тестовый датасет:

In [19]:
path_to_test = r'data/prepared_data/GSD/gsd_test_data.csv'

In [21]:
old_df2 = pd.read_csv(path_to_test)
forms2 = old_df2['form'].to_list()
old_df2

,form,data
0,Билли,"Билли,PROPN,Anim,Nom,Masc,Sing"
1,начал,"начать,VERB,Perf,Masc,Ind,Sing,Past,Fin,Act"
2,играть,"играть,VERB,Imp,Inf,Act"
3,за,"за,ADP,None"
4,резервный,"резервный,ADJ,Inan,Acc,Pos,Masc,Sing"
...,...,...
11380,Землю,"земля,NOUN,Inan,Acc,Fem,Sing"
11381,ради,"ради,ADP,None"
11382,получения,"получение,NOUN,Inan,Gen,Neut,Sing"
11383,энергии,"энергия,NOUN,Inan,Gen,Fem,Sing"


In [23]:
%%time
predictions2 = []
for form in forms2:
    pred = ppln({"form": form}, do_sample=True)
    print(pred)
    predictions2.append(pred)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'PROPN,Anim,Nom,Masc,Sing'}]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'VERB,Perf,Masc,Ind,Sing,Past,Fin,Act'}]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

[{'generated_text': 'ADP,None'}]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'NOUN,Inan,Gen,Neut,Sing'}]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'NOUN,Inan,Gen,Fem,Sing'}]
[{'generated_text': 'PUNCT,None'}]
CPU times: total: 3d 4h 7min 12s
Wall time: 10h 24min 4s


In [24]:
old_df2['predicted_data'] = predictions2

Записываем в новый файл в формате "слово - истинные значения - предсказанные значения":

In [25]:
old_df2.to_csv(r'testing/gsd_test_rupr_no_cl.csv')